In [3]:
import torch
import copy
import torch.nn.functional as F
from models.tiny_vgg import TinyVGG
from models.module import QLinear
def cmp(a,b):
    return (a-b).abs().max()
model = TinyVGG(image_size=32,num_classes=10)
model.eval()
model.fuse_bn()
model.quantize(w_num_bits=4,a_num_bits=4)
model.load_state_dict(torch.load('./qat_logs/cifar10/tiny_vgg_w4a4/checkpoint_max.pth')['net'])
q_model = copy.deepcopy(model)
q_model.freeze()


q_model.fc1.linear_module.bias

/tmp/ipykernel_431389/3080446895.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./qat_logs/cifar10/tiny_vgg_w4a4/checkpoint_max.pth')

Parameter containing:
tensor([0., 0., 0.,  ..., 0., 0., 7.], requires_grad=True)

In [8]:
q_model.fc1.linear_module.bias.min()

tensor(-11., grad_fn=<MinBackward1>)

In [2]:
dummy_input = model.layer4[-1].add.qo.fake_quantize(torch.rand([1000,8192]))
q_input = q_model.layer4[-1].qo.quantize_tensor(dummy_input)

out = model.linear(dummy_input, qi=q_model.layer4[-1].qo)
q_out = q_model.linear.qo.dequantize_tensor(q_model.linear.quantize_inference(q_input))
cmp(out, q_out)

tensor(0.9553, grad_fn=<MaxBackward1>)

In [3]:
x = q_model.layer4[-1].add.qo.fake_quantize(dummy_input)
(x==dummy_input).sum()


tensor(8192000)

In [4]:
q_model.linear

tensor([[-7.6420, -6.6868, -6.6868, -7.6420, -7.6420, -7.6420, -7.6420, -7.6420,
         -7.6420, -7.6420]], grad_fn=<MulBackward0>)

In [5]:
q_out

tensor([[-7.6420, -6.6868, -5.7315, -7.6420, -7.6420, -6.6868, -7.6420, -7.6420,
         -7.6420, -7.6420]], grad_fn=<MulBackward0>)

In [14]:
q_model.linear.qo

QParam()

In [5]:
q_model.layer1[0].conv1.n

tensor(10.)